In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import os
import wandb
import yaml

In [ ]:
conf_file = "config/config-seg.yaml"
# conf_file = "config/config-seg-finetune.yaml"
# conf_file = "config/config-seg-joint.yaml"

with open(conf_file) as f:
    conf_init = yaml.load(f)

In [ ]:
name = 'tertiary'

In [ ]:
#os.environ['WANDB_MODE'] = 'dryrun'
wandb.init(project=conf_init['project'], config=conf_init, name=name)
conf = wandb.config

In [ ]:
from pprint import pprint
pprint(dict(conf))

In [ ]:
from glob import glob
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [ ]:
train_dir = '../../datasets/xview/train'
test_dir = '../../datasets/xview/test'
suppl_dir = '../../datasets/xview/tier3'
tertiary_dir = '../../datasets/spacenet'

In [ ]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob),
])

In [ ]:
import segmentation_models_pytorch as smp

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}

model_classes = conf.nclasses

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
class DualWrapper(nn.Module):
    def __init__(self, m, channels_in=16):
        super().__init__()
        self._model = m
        
        self.pre_mixin = nn.Sequential(
            nn.Conv2d(channels_in, channels_in, kernel_size=1),
            nn.BatchNorm2d(channels_in),
            nn.ReLU()
        )
        
        self.post_mixin = nn.Sequential(
            nn.Conv2d(channels_in, channels_in, kernel_size=1),
            nn.BatchNorm2d(channels_in),
            nn.ReLU()
        )
        
    def forward(self, x):
        pre, post = x[:,:3], x[:,3:]
        
        pre_out = self.pre_mixin(self._model.decoder(*self._model.encoder(pre)))
        post_out = self.post_mixin(self._model.decoder(*self._model.encoder(post)))
        
        return self._model.segmentation_head(pre_out + post_out)

In [ ]:
from xv.nn.layers import FrozenBatchNorm2d

model = segmentation_types[conf.segmentation_arch](
    conf.encoder,
    classes=model_classes,
    decoder_attention_type=conf.attention,
)

if conf.mode == "dual":
    model = DualWrapper(model)

if conf.load_weights:
    state_dict = torch.load(conf.load_weights)
    print(model.load_state_dict(state_dict))

preprocess_fn = get_preprocessing_fn(conf.encoder)

if conf.freeze_encoder_norm:
    model.encoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.encoder)
    
if conf.freeze_decoder_norm:
    model.decoder = FrozenBatchNorm2d.convert_frozen_batchnorm(model.decoder)
    
model = model.cuda()

In [ ]:
from xv.nn.oc import oc

class OCSegment(nn.Module):
    def __init__(self, model, decoder_channels=128):
        super().__init__()
        self._model = model
        self.context = oc.InterlacedSparseSelfAttention(decoder_channels)
        
    def forward(self, x):
        features = self._model.encoder(x)
        out = self._model.decoder(*features)
        out = self.context(out)
        return self._model.segmentation_head(out)
        
        
#model = OCSegment(model).cuda()

In [ ]:
import random
from xv import dataset
import pandas as pd

train_stems = pd.read_csv('config/train_stems.csv', header=None)[0]
dev_stems = pd.read_csv('config/dev_stems.csv', header=None)[0]

train_files = [f'{train_dir}/labels/{stem}_{conf.data_prefix}_disaster.json' for stem in train_stems]
dev_files = [f'{train_dir}/labels/{stem}_{conf.data_prefix}_disaster.json' for stem in dev_stems]

train_instances = dataset.get_instances(train_files, filter_none=conf.filter_none)
dev_instances = dataset.get_instances(dev_files, filter_none=conf.filter_none)

len(train_instances), len(dev_instances)

In [ ]:
if conf.add_suppl:
    train_instances *= conf.train_repeat
    suppl_files = glob(f'{suppl_dir}/labels/*{conf.data_prefix}_disaster.json')
    suppl_instances = dataset.get_instances(suppl_files, filter_none=conf.filter_none)
    train_instances += suppl_instances
    print(len(train_instances))

In [ ]:
if conf.add_tertiary:
    import pandas as pd
    from PIL import Image
    import shapely
    tertiary_instances = []

    for subdir in tqdm(os.listdir(tertiary_dir)):
        csv = glob(f'{tertiary_dir}/{subdir}/summaryData/*.csv')[0]    
        df = pd.read_csv(csv)
        for img, group in tqdm(df.groupby('ImageId')):
            filepath = f'{tertiary_dir}/{subdir}/rgb/rgb_{img}.jpg'
            _im = Image.open(filepath)
            h,w = _im.size
            polys = [shapely.wkt.loads(p) for p in group.PolygonWKT_Pix]
            points_list = [list(poly.exterior.coords) for poly in polys if poly.exterior is not None]
            points_list = [[(x/w, y/h) for x,y,_ in points] for points in points_list]
            annotations = []
            for points in points_list:
                x,y = zip(*points)
                annotations.append({
                    'bbox':[(min(x), min(y)), (max(x), max(y))],
                    'bbox_mode': None,
                    'segmentation': points,
                    'category_id': 0,
                    'iscrowd': 0  
                })

            tertiary_instances.append({
                'height': h,
                'width': w,
                'file_name': filepath,
                'annotations': annotations,
            })
    train_instances += tertiary_instances
    print(len(train_instances))

In [ ]:
from pytorch_toolbelt import losses
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss
loss_dict

In [ ]:
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss

if 'class_weight' in dict(conf):
    weights = torch.Tensor(conf.class_weight).float().cuda()
    loss = CrossEntropyLoss(weights)
else:
    loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [ ]:
import apex

optims = {
    'adam': torch.optim.Adam,
    'sgd': torch.optim.SGD
}

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [ ]:
from xv import dataset
train_dataset = dataset.BuildingSegmentationDataset(
    instances=train_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=augment,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

dev_dataset = dataset.BuildingSegmentationDataset(
    instances=dev_instances,
    nclasses=conf.nclasses,
    resolution=conf.training_resolution,
    augment=None,
    preprocess_fn=preprocess_fn,
    mode=conf.mode,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=False,
    num_workers=10,
)

In [ ]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level);

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience
)

In [ ]:
from torchvision.ops import misc as misc_nn_ops
import random

class MultiScaleResize(nn.Module):
    def __init__(self, mode="categorical", scales = (0.5, 0.75, 1.)):
        super().__init__()
        self.mode = mode
        self.scales = scales
    
    @torch.no_grad()
    def forward(self, batch):
        scale = random.choice(self.scales)
        if scale == 1.:
            return batch
        if self.mode is None or self.mode == "dual":
            im, mask = batch
            mask_dtype = mask.dtype
            im = torch.nn.functional.interpolate(im, scale_factor=scale, mode='bilinear', align_corners=False)
            mask = misc_nn_ops.interpolate(mask.float(), scale_factor=scale).to(mask_dtype)
            return im, mask
        if self.mode == "categorical":
            im, (damage_mask, damage) = batch
            dmg_msk_dtype = damage_mask.dtype()
            dmg_dtype = damage.dtype()
            im = torch.nn.functional.interpolate(im, scale_factor=scale, mode='bilinear', align_corners=False)
            damage_mask = misc_nn_ops.interpolate(damage_mask[None].float(), scale_factor=scale)[0].to(dmg_msk_dtype)
            damage_one_hot = torch.nn.functional.one_hot(damage).permute(0,3,1,2)
            damage = misc_nn_ops.interpolate(damage_one_hot.float(), scale_factor=scale).argmax(1)
            return im, (damage_mask, damage)

In [ ]:
train_resize = MultiScaleResize(conf.mode, conf.training_scales)

In [ ]:
best_score = 0
epoch = 0

In [ ]:
from xv import run

train_fn = run.train_segment if conf.nclasses == 1 else run.train_damage
eval_fn = run.evaluate_segment if conf.nclasses == 1 else run.evaluate_damage

for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = train_fn(model, optim, train_loader, loss, train_resize=train_resize, mode=conf.mode)
    metrics.update(train_metrics)
    
    dev_metrics = eval_fn(model, dev_loader, loss, mode=conf.mode)
    metrics.update(dev_metrics)
    
    if conf.mode != "dual":
        examples = run.sample_masks(model, dev_instances, preprocess_fn, n=3)
        metrics['examples'] = [wandb.Image(im, caption=f'mask:{ix}') for e in examples for ix, im in enumerate(e)]
    
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    score = metrics[conf.metric]

    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score

## 

In [ ]:
if False:
    rundir = 'b-' + wandb.run.dir.split('/')[-1]
    !mkdir ../weights/{rundir}
    !cp {os.path.join(wandb.run.dir, "state_dict.pth")}  ../weights/{rundir}
    import json
    with open(f'../weights/{rundir}/conf.json', 'w') as f:
        json.dump(dict(conf), f)

In [ ]:
print(rundir)
!ls -lah ../weights/{rundir}